In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

In [10]:
# Load your dataset
df = pd.read_csv('../data/creditcard_2023.csv')
smalldf = df.sample(n=10000)
X_imbalanced = smalldf.drop(['id', 'Class'], axis=1).values  # Drop 'id' and 'Class' columns
y_imbalanced = smalldf['Class'].values
X_train = X_imbalanced[y_imbalanced == 1]
print(X_train.shape)

(4965, 29)


In [79]:
dframe = pd.DataFrame(X_train)

In [80]:
dframe.describe()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
count,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,...,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000,49823.000000
mean,-0.506762,0.491202,-0.680050,0.734459,-0.342497,-0.433234,-0.492767,0.150925,-0.588744,-0.672569,...,0.183441,0.099624,0.023144,0.005005,-0.125367,0.065341,0.069587,0.209760,0.103910,12101.492340
std,0.901217,1.013999,0.760846,0.683609,1.158574,1.042189,1.026723,1.373159,0.882014,0.828765,...,1.133364,1.369469,1.242421,1.256476,0.831236,1.133747,0.907394,1.286648,1.161124,6896.257558
min,-2.637362,-4.215050,-2.518046,-1.369576,-3.301131,-3.988017,-4.351283,-6.532979,-3.750965,-3.163128,...,-4.860277,-12.165524,-6.450353,-14.452018,-3.060251,-6.810179,-3.134266,-4.947034,-4.678912,50.950000
25%,-0.992557,-0.062883,-1.059735,0.219389,-0.896395,-1.055923,-0.996565,-0.171379,-1.130059,-1.098828,...,-0.294850,-0.054332,-0.457349,-0.313174,-0.685505,-0.500830,-0.521169,-0.249677,-0.263142,6168.195000
50%,-0.470142,0.314712,-0.640461,0.680332,-0.221058,-0.337221,-0.272772,-0.017675,-0.531379,-0.590933,...,0.115143,0.124852,-0.017050,-0.090103,-0.108883,0.029291,0.055095,0.263958,0.240575,12117.570000
75%,0.018500,0.856295,-0.190826,1.216267,0.282711,0.206490,0.148619,0.264065,0.006657,-0.222277,...,0.606533,0.360037,0.470965,0.211185,0.444517,0.611649,0.727193,0.750970,0.746312,18100.805000
max,1.845476,4.361219,2.192263,2.394164,9.193748,3.529467,3.830865,5.956513,3.325877,3.380819,...,9.276019,8.087080,9.768747,6.553262,2.442869,3.836984,4.622883,4.834397,4.147711,24039.480000


In [4]:
def build_discriminator(input_dim):
    model = Sequential([
        # Input layer with 128 units
        layers.Input(shape=(input_dim,)),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        
        # Second hidden layer with 64 units
        layers.Dense(128),
        layers.LeakyReLU(alpha=0.2),
        
        # Third hidden layer with 32 units
        layers.Dense(64),
        layers.LeakyReLU(alpha=0.2),
        
        # Output layer with a single unit (real or fake)
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

# Example usage:
input_dim = 29  # Number of features in the dataset
discriminator = build_discriminator(input_dim)

# Compile the model
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

# Print the model summary
discriminator.summary()


C:\Users\mirot\Documents\Data_Science\AI_Data_Augmentation\env\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │           7,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 48,897 (191.00 KB)

 Trainable params: 48,897 (191.00 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
def build_generator(latent_dim, output_dim):
    model = Sequential([
        # Input layer (latent space)
        layers.Input(shape=(latent_dim,)),
        
        # First hidden layer with 64 units
        layers.Dense(64),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        
        # Second hidden layer with 128 units
        layers.Dense(128),
        layers.LeakyReLU(alpha=0.2),
        
        # Third hidden layer with 256 units
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8),
        
        # Output layer with 'output_dim' units and linear activation
        layers.Dense(output_dim)
    ])
    
    return model

# Example usage:
latent_dim = 100  # Size of the latent space (can be adjusted)
output_dim = 29   # Number of features in the generated data
generator = build_generator(latent_dim, output_dim)

# Print the model summary
generator.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 64)                  │           6,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 29)                  │           7,453 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 56,541 (220.86 KB)

 Trainable params: 55,901 (218.36 KB)

 Non-trainable params: 640 (2.50 KB)

In [8]:
from tensorflow.keras import backend as K

def clear_session():
    K.clear_session()

In [11]:
# Define the binary cross-entropy loss
bce_loss = tf.keras.losses.BinaryCrossentropy()

# Define the optimizers
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.5)

def train_gan(generator, discriminator, generator_optimizer, discriminator_optimizer, latent_dim, real_data, epochs, batch_size):
        # Define labels
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))
    epoch_accuracy = []
    epoch_precision = []
    epoch_recall = []
    epoch_f1 = []

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs} started.")
        for batch in range(len(real_data) // batch_size):
            print(f"Batch {batch + 1} of Epoch {epoch + 1} processing.")
            # Train the discriminator
            
            # 1. Sample random noise and generate fake data
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            generated_data = generator.predict(noise)
            
            # 2. Get a batch of real data
            real_data_batch = real_data[batch * batch_size: (batch + 1) * batch_size]
            
            # 3. Train the discriminator on real and fake data
            d_loss_real = discriminator.train_on_batch(real_data_batch, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)
            
            # 4. Compute the total discriminator loss
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            pred_real = discriminator.predict(real_data_batch)
            pred_fake = discriminator.predict(generated_data)
            
            # Threshold predictions at 0.5 to get binary labels
            pred_real_labels = (pred_real > 0.5).astype(int)
            pred_fake_labels = (pred_fake > 0.5).astype(int)
            
            # Concatenate real and fake predictions for metric calculation
            y_true = np.concatenate([real_labels, fake_labels])
            y_pred = np.concatenate([pred_real_labels, pred_fake_labels])
            
            # Calculate and store metrics
            batch_accuracy = accuracy_score(y_true, y_pred)
            batch_precision = precision_score(y_true, y_pred)
            batch_recall = recall_score(y_true, y_pred)
            batch_f1 = f1_score(y_true, y_pred)
            
            epoch_accuracy.append(batch_accuracy)
            epoch_precision.append(batch_precision)
            epoch_recall.append(batch_recall)
            epoch_f1.append(batch_f1)
            
            # Train the generator (to fool the discriminator)
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, real_labels)
        
        # Calculate average metrics for the epoch
        avg_accuracy = np.mean(epoch_accuracy)
        avg_precision = np.mean(epoch_precision)
        avg_recall = np.mean(epoch_recall)
        avg_f1 = np.mean(epoch_f1)
        
        print(f"Epoch {epoch + 1}/{epochs} | Discriminator Loss: {d_loss} | Generator Loss: {g_loss}")
        print(f"Accuracy: {avg_accuracy:.4f} | Precision: {avg_precision:.4f} | Recall: {avg_recall:.4f} | F1-score: {avg_f1:.4f}")
        clear_session()
# Example usage:
latent_dim = 100  # Size of the latent space
batch_size = 64
epochs = 10
real_data = X_train#real_data = np.array(...)  # Your real dataset here

# Combine the generator and discriminator into a GAN model
gan_input = tf.keras.Input(shape=(latent_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)
gan = tf.keras.Model(gan_input, gan_output)

# Compile the GAN model
gan.compile(optimizer=generator_optimizer, loss=bce_loss)
discriminator.compile(optimizer=discriminator_optimizer, loss=bce_loss, metrics=['accuracy'])

# Train the GAN
train_gan(generator, discriminator, generator_optimizer, discriminator_optimizer, latent_dim, real_data, epochs, batch_size)
# Save the generator model
generator.save('generator_model.keras')

# Save the discriminator model
discriminator.save('discriminator_model.keras')

# Save the GAN model (if desired)
gan.save('gan_model.keras')


Epoch 1/10 started.
Batch 1 of Epoch 1 processing.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step   
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Batch 2 of Epoch 1 processing.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Batch 3 of Epoch 1 processing.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Batch 4 of Epoch 1 processing.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
Batch 5 of Epoch 1 processing.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Batch 6 of Epoch 1 processing.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
Batch 7 of Epoch 1 processing.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2

In [62]:
# Generate synthetic data
num_samples = 10000
random_latent_vectors = np.random.normal(0,1,size=(num_samples, latent_dim))
synthetic_data_scaled = generator.predict(random_latent_vectors)

# Inverse transform to original scale
synthetic_data = scaler.inverse_transform(synthetic_data_scaled)
df_synthetic = pd.DataFrame(synthetic_data, columns=df.columns[1:-1])
df_synthetic['Class'] = 1


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [59]:
# Combine with the original dataset
df_augmented = pd.concat([df, df_synthetic])

# Verify the new class distribution
print("Augmented class distribution:")
print(df_augmented['Class'].value_counts())

# Save the augmented dataset
df_augmented.to_csv('augmented_data.csv', index=False)


Augmented class distribution:
Class
0    284315
1     38432
Name: count, dtype: int64


In [13]:
# Save the generator model
generator.save('generator_model.keras')

# Save the discriminator model
discriminator.save('discriminator_model.keras')

# Save the GAN model (if desired)
gan.save('gan_model.keras')


In [66]:
# Load your dataset
df = pd.read_csv('../data/creditcard_2023.csv')

X_imbalanced = df.drop(['id', 'Class'], axis=1).values  # Drop 'id' and 'Class' columns
y_imbalanced = df['Class'].values
X_train_full = X_imbalanced[y_imbalanced == 1]
df_fraud = pd.DataFrame(X_train_full,columns = df.columns[1:-1])
# Normalize the data
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_imbalanced = scaler.fit_transform(X_imbalanced)

# X_train = X_imbalanced[y_imbalanced == 1][:10000]

In [7]:
loaded_model = keras.saving.load_model("generator_model.keras")